In [1]:
import pathlib
import pandas as pd
from odapi.connectors import Irceline
from odapi.toolbox.statistics import StatisticalTest 
from odapi.settings import settings
settings.logger.setLevel(50)

In [2]:
client = Irceline()

In [3]:
files = list(pathlib.Path("./data").glob("*.pickle"))

In [4]:
dfs = []
for file in files:
    dfs.append(pd.read_pickle(file))
data = pd.concat(dfs)

In [5]:
data

,BC/41N043 (µg/m³),BC/41R001 (µg/m³),BC/41R002 (µg/m³),BC/41R012 (µg/m³),BC/41WOL1 (µg/m³),BZN/41B006 (µg/m³),CO/41B001 (mg/m³),CO/41B004 (mg/m³),CO/41B006 (mg/m³),CO/41B008 (mg/m³),...,SO2/41WOL1 (µg/m³),T/41R001 (°C),T/41R012 (°C),TOL/41B006 (µg/m³),WD/41R001 (°G),WD/41R012 (°G),WS/41R001 (m/s),WS/41R012 (m/s),RH/41R001 (%),RH/41R012 (%)
start,,,,,,,,,,,,,,,,,,,,,
2017-12-31 23:00:00+00:00,1.305,0.350,0.465,0.125,0.200,NaN,NaN,0.210,NaN,NaN,...,8.0,8.70,8.20,NaN,210.0,219.0,8.265,8.215,NaN,NaN
2018-01-01 00:00:00+00:00,0.570,0.245,0.515,0.100,0.200,NaN,0.125,0.200,NaN,NaN,...,6.5,8.35,8.25,NaN,207.0,216.0,9.060,9.410,NaN,NaN
2018-01-01 01:00:00+00:00,0.390,0.255,0.470,0.090,0.180,NaN,0.125,0.190,NaN,NaN,...,5.0,8.05,8.10,NaN,216.0,224.0,9.130,9.050,NaN,NaN
2018-01-01 02:00:00+00:00,0.145,0.180,0.410,0.050,0.115,NaN,0.120,0.165,NaN,NaN,...,4.5,7.45,7.20,NaN,216.0,224.0,9.200,8.615,NaN,NaN
2018-01-01 03:00:00+00:00,0.170,0.150,0.370,0.035,0.070,NaN,0.120,0.160,NaN,NaN,...,6.0,6.85,6.65,NaN,212.0,223.0,8.685,8.650,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-31 19:00:00+00:00,6.655,NaN,1.485,0.670,0.685,0.050,0.530,0.335,0.420,0.530,...,NaN,32.80,NaN,13.5,134.0,143.0,2.175,2.745,27.0,37.0
2020-07-31 20:00:00+00:00,3.695,NaN,NaN,1.110,1.035,0.045,0.325,0.300,0.395,0.375,...,NaN,30.90,NaN,11.0,203.0,224.0,2.650,2.260,33.0,38.5
2020-07-31 21:00:00+00:00,1.920,NaN,1.165,0.785,0.790,0.030,0.330,0.245,0.385,0.300,...,NaN,29.60,NaN,10.0,244.0,272.0,1.460,1.835,38.5,40.0


In [6]:
t0 = "2019-01-01"
t1 = "2020-01-01"
t2 = "2020-03-18"
t3 = "2020-04-05"

In [7]:
ref = data.loc[t0:t1,:]
exp = data.loc[t2:t3,:]

In [8]:
xr = ref.filter(regex="NO2/41(R002|R012)")
xe = exp.filter(regex="NO2/")

In [9]:
r1 = StatisticalTest.apply(xr, xe, test="T-Test", mode="product")
r2 = StatisticalTest.apply(xr, xe, test="KS-Test", mode="product")

In [10]:
r = pd.concat([r1, r2])

In [11]:
rc = r.pivot_table(index="exp_key", columns=["ref_key", "test"], values=["statistic", "pvalue"])

In [12]:
rc

pvalue                                    \
ref_key            NO2/41R002 (µg/m³)                NO2/41R012 (µg/m³)   
test                          KS-Test         T-Test            KS-Test   
exp_key                                                                   
NO2/41B001 (µg/m³)       6.638004e-16   2.092605e-12       2.140410e-45   
NO2/41B004 (µg/m³)       7.237567e-53   2.392048e-44       4.259126e-16   
NO2/41B006 (µg/m³)       4.549169e-58   1.878286e-75       4.512982e-06   
NO2/41B008 (µg/m³)       1.062898e-03   2.039196e-06       6.012615e-91   
NO2/41B011 (µg/m³)      1.482474e-132  5.945548e-151       5.370494e-19   
NO2/41MEU1 (µg/m³)       2.356350e-75   3.232212e-82       9.001963e-02   
NO2/41N043 (µg/m³)       1.827222e-53   4.026305e-36       4.236797e-04   
NO2/41R001 (µg/m³)       3.449438e-32   5.443171e-38       8.780941e-17   
NO2/41R002 (µg/m³)       2.265232e-68   5.711683e-88       4.653371e-08   
NO2/41R012 (µg/m³)      1.781916e-145  5.803835e-202       9.801520e-20   
NO2/41WOL1 (µg/m³)       4.722973e-48   4.782063e-51       2.513346e-06   

                                          statistic             \
ref_key                          NO2/41R002 (µg/m³)              
test                      T-Test            KS-Test     T-Test   
exp_key                                                          
NO2/41B001 (µg/m³)  1.112009e-44           0.202755   7.203419   
NO2/41B004 (µg/m³)  4.801756e-14           0.373196  15.402445   
NO2/41B006 (µg/m³)  6.282255e-07           0.391268  21.616458   
NO2/41B008 (µg/m³)  3.973344e-76           0.093256  -4.807379   
NO2/41B011 (µg/m³)  1.030293e-13           0.592381  36.445864   
NO2/41MEU1 (µg/m³)  6.987125e-02           0.446092  23.058701   
NO2/41N043 (µg/m³)  9.428201e-09           0.375318  13.617912   
NO2/41R001 (µg/m³)  8.451126e-19           0.290444  13.996877   
NO2/41R002 (µg/m³)  5.577663e-06           0.424797  23.955574   
NO2/41R012 (µg/m³)  1.682705e-25           0.620668  45.308459   
NO2/41WOL1 (µg/m³)  2.160131e-07           0.355653  16.928613   

                                                  
ref_key            NO2/41R012 (µg/m³)             
test                          KS-Test     T-Test  
exp_key                                           
NO2/41B001 (µg/m³)           0.345592 -15.604294  
NO2/41B004 (µg/m³)           0.204013  -7.770319  
NO2/41B006 (µg/m³)           0.122459  -5.048026  
NO2/41B008 (µg/m³)           0.490317 -22.420211  
NO2/41B011 (µg/m³)           0.222084   7.650382  
NO2/41MEU1 (µg/m³)           0.059804  -1.816727  
NO2/41N043 (µg/m³)           0.098780  -5.845757  
NO2/41R001 (µg/m³)           0.208429  -9.233501  
NO2/41R002 (µg/m³)           0.142382  -4.591414  
NO2/41R012 (µg/m³)           0.226458  10.997553  
NO2/41WOL1 (µg/m³)           0.125186  -5.264519

In [13]:
rc.reorder_levels([1,2,0], axis=1).sort_index(axis=1)

ref_key            NO2/41R002 (µg/m³)                                      \
test                          KS-Test                   T-Test              
                               pvalue statistic         pvalue  statistic   
exp_key                                                                     
NO2/41B001 (µg/m³)       6.638004e-16  0.202755   2.092605e-12   7.203419   
NO2/41B004 (µg/m³)       7.237567e-53  0.373196   2.392048e-44  15.402445   
NO2/41B006 (µg/m³)       4.549169e-58  0.391268   1.878286e-75  21.616458   
NO2/41B008 (µg/m³)       1.062898e-03  0.093256   2.039196e-06  -4.807379   
NO2/41B011 (µg/m³)      1.482474e-132  0.592381  5.945548e-151  36.445864   
NO2/41MEU1 (µg/m³)       2.356350e-75  0.446092   3.232212e-82  23.058701   
NO2/41N043 (µg/m³)       1.827222e-53  0.375318   4.026305e-36  13.617912   
NO2/41R001 (µg/m³)       3.449438e-32  0.290444   5.443171e-38  13.996877   
NO2/41R002 (µg/m³)       2.265232e-68  0.424797   5.711683e-88  23.955574   
NO2/41R012 (µg/m³)      1.781916e-145  0.620668  5.803835e-202  45.308459   
NO2/41WOL1 (µg/m³)       4.722973e-48  0.355653   4.782063e-51  16.928613   

ref_key            NO2/41R012 (µg/m³)                                     
test                          KS-Test                  T-Test             
                               pvalue statistic        pvalue  statistic  
exp_key                                                                   
NO2/41B001 (µg/m³)       2.140410e-45  0.345592  1.112009e-44 -15.604294  
NO2/41B004 (µg/m³)       4.259126e-16  0.204013  4.801756e-14  -7.770319  
NO2/41B006 (µg/m³)       4.512982e-06  0.122459  6.282255e-07  -5.048026  
NO2/41B008 (µg/m³)       6.012615e-91  0.490317  3.973344e-76 -22.420211  
NO2/41B011 (µg/m³)       5.370494e-19  0.222084  1.030293e-13   7.650382  
NO2/41MEU1 (µg/m³)       9.001963e-02  0.059804  6.987125e-02  -1.816727  
NO2/41N043 (µg/m³)       4.236797e-04  0.098780  9.428201e-09  -5.845757  
NO2/41R001 (µg/m³)       8.780941e-17  0.208429  8.451126e-19  -9.233501  
NO2/41R002 (µg/m³)       4.653371e-08  0.142382  5.577663e-06  -4.591414  
NO2/41R012 (µg/m³)       9.801520e-20  0.226458  1.682705e-25  10.997553  
NO2/41WOL1 (µg/m³)       2.513346e-06  0.125186  2.160131e-07  -5.264519